# CyJS Assembler

* Assemble a network from text using existing INDRA functionality
* Translate this into a model supported by cytoscape.js
* Visualize standalone, or within notebook
---

### I. Describe your network in plain English

* We have described the interactions in the McCormick RAS pathway map using plain English.
* Let's load this set of sentences.

from __future__ import absolute_import, print_function, unicode_literals
from builtins import dict, str
statement_path = 'ras_pathway.txt'
txt = open(statement_path, 'rt').read()
print(txt)

---
### II. Process text into INDRA statements using REACH parser

* If we have never parsed this text before (or we've made changes to it) we should process it with REACH
* This will yield a set of INDRA statements
* We will then pickle the statements so we don't have to parse them every time we run this script.
    * Change the cell below to from Code -> Raw NBConvert in order to not execute it every time

In [ ]:
txt = 'ras_pathway.txt'

In [ ]:
from indra import reach
rp = reach.process_text(txt, offline=False)
st = rp.statements

In [ ]:
from indra.preassembler.grounding_mapper import GroundingMapper, gm

GM = GroundingMapper(gm)
mapped_stmts = GM.map_agents(st, do_rename=True)

import pickle
pickle.dump(mapped_stmts, open("st.pkl", "wb"))

---
### III. Assemble an INDRA model

In [ ]:
import pickle
st = pickle.load(open("st.pkl", "rb"))

In [ ]:
from indra.statements import *
from indra.assemblers import CyJSAssembler

In [ ]:
import pandas as pd

In [ ]:
ccle = pd.read_table("CCLE_sample_info_file_2012-10-18.txt")

In [ ]:
cell_lines = ccle['CCLE name'].tolist()

In [ ]:
for line in cell_lines:
    cja = CyJSAssembler()
    cja.add_statements(st)
    cja.make_model(grouping=True, 
                   drop_virtual_edges=False,
                   add_edge_weights=True)

    cja.set_context(cell_type = line, 
                    bin_expression = True, 
                    n_bins = 9)
    
    if (len(cja._exp_colorscale) == 0):
        print('ignore cell line: ' + line)
        continue
 
    cja.save_json('static/cyjs/model_'+ line + '.json')
    #cja.save_model('../../indra/models/ras_pathway/cyjs/model.js')

In [ ]:
files = os.listdir('static/cyjs/')

In [ ]:
ccle_dict = {x[6:len(x)-5]:x for x in files if 'model' in x}

In [ ]:
ccle_dict

In [ ]:
pwd

In [ ]:
import json

In [ ]:
ccle_dict_json = json.dumps(ccle_dict, indent=1, sort_keys=True)

In [ ]:
ccle_dict_json

In [ ]:
 with open('static/cell_dict.json', 'wt') as fh:
        fh.write(ccle_dict_json)